In [70]:
%load_ext autoreload
%autoreload 2
%aimport -jax
%aimport -jaxlib
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(),"..","projects")))
print(sys.path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
['/Users/daniel/Documents/code/stable_imitation/stanza/notebooks', '/opt/homebrew/Cellar/python@3.11/3.11.3/Frameworks/Python.framework/Versions/3.11/lib/python311.zip', '/opt/homebrew/Cellar/python@3.11/3.11.3/Frameworks/Python.framework/Versions/3.11/lib/python3.11', '/opt/homebrew/Cellar/python@3.11/3.11.3/Frameworks/Python.framework/Versions/3.11/lib/python3.11/lib-dynload', '', '/Users/daniel/Documents/code/stable_imitation/stanza/.venv/lib/python3.11/site-packages', '/Users/daniel/Documents/code/stable_imitation/stanza', '/Users/daniel/Documents/code/stable_imitation/stanza/projects', '/Users/daniel/Documents/code/stable_imitation/stanza/projects', '/Users/daniel/Documents/code/stable_imitation/stanza/projects', '/Users/daniel/Documents/code/stable_imitation/stanza/projects']


In [77]:
import jax
import jax.numpy as jnp
from stanza.data import Data, PyTreeData
from stanza.util import shape_tree

# dummy data
train_data = Data.from_pytree((jnp.ones((100,32,32,3)), jnp.ones((100, 10))))
test_data = Data.from_pytree((jnp.ones((100,32,32,3)), jnp.ones((100, 10))))
# cifar data
# from stanza.data.cifar import cifar10
# train_data, test_data = cifar10()
# def map_fn(sample):
#     x, y = sample
#     x = x.astype(float) / 255
#     y = jax.nn.one_hot(y, 10)
#     return x, y
# train_data = PyTreeData.from_data(train_data.map(map_fn))
# test_data = PyTreeData.from_data(test_data.map(map_fn))

In [78]:
total_steps = 1000

In [79]:
import haiku as hk
from jax.random import PRNGKey
import jax
import optax
from stanza.util.random import PRNGSequence
from stanza.util import shape_tree
import rich.progress as progress
from stanza.train.rich import MofNColumn
from rich.style import Style

def run(use_sam=False):
    def net(x):
        flat, _ = jax.flatten_util.ravel_pytree(x)
        mlp = hk.nets.MLP([128,64, 32, 1])
        y = mlp(flat).squeeze()
        return y

    net = hk.transform(net)
    init_params = net.init(PRNGKey(300), train_data[0][0])

    optim = optax.adam(optax.cosine_decay_schedule(1e-3, total_steps))
    sub_optim = optax.scale(1e-5)

    params = init_params
    opt_state = optim.init(init_params)
    sub_opt_state = optim.init(init_params)

    @jax.jit
    def loss_fn(params, batch):
        x, y = batch
        y_pred = net.apply(params, None, x)
        return jnp.sum(jnp.square(y_pred - y)) 

    @jax.jit
    def batch_loss_fn(params, batch):
        s = jax.vmap(loss_fn, in_axes=(None, 0))(params, batch)
        loss = jnp.mean(s)
        return loss, loss

    batch_loss_grad = jax.grad(batch_loss_fn, argnums=0, has_aux=True)

    rng = PRNGSequence(42)
    with progress.Progress(
                progress.TextColumn("[progress.description]{task.description}"),
                progress.BarColumn(finished_style=Style(color="green")),
                MofNColumn(),
                progress.TimeRemainingColumn(),
                progress.TimeElapsedColumn(),
                progress.TextColumn("loss={task.fields[loss]}"),
            ) as pbar:
        task = pbar.add_task("", total=total_steps,loss=0.)
        for i in range(total_steps):
            batch = train_data.sample_batch(32, next(rng))
            # do a backwards step
            if use_sam:
                grad, loss = batch_loss_grad(params, batch)
                updates, sub_opt_state = sub_optim.update(
                    grad, sub_opt_state, params)
                sub_params = optax.apply_updates(params, updates)
            else:
                sub_params = params

            # do the main step with gradient calculated at the perturbed
            # point
            grad, loss = batch_loss_grad(sub_params, batch)
            updates, opt_state = optim.update(grad, opt_state, params)
            params = optax.apply_updates(params, updates)
            pbar.update(task, advance=1., loss=loss.item())

In [80]:
run(use_sam=True)
run(use_sam=False)


Output()

Output()